In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-2sp_bc23
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-2sp_bc23


In [ ]:
!pip install -q transformers

In [ ]:
from genericpath import exists
import sys
import os

my_path = '/content/bertsum'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/MyDrive/ColabNotebooks/dacon-gas/AI/04/bertsum_realization', my_path)
sys.path.insert(0, my_path)

FileExistsError: ignored

In [ ]:
!pip install tensorboardX pyrouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import transformers

import datetime
import easydict
import itertools
import json
import matplotlib
import pathlib
import pprint
import re

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from operator import itemgetter
from pathlib import Path
from tqdm import tqdm
from typing import List, Dict

## Version.
print("[VERSION]")
print(f"torch: {torch.__version__}")
print(f"transformers: {transformers.__version__}")

[VERSION]
torch: 1.12.0+cu113
transformers: 4.20.1


In [ ]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
except:
    raise AssertionError()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -V

Python 3.7.13


In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   40G   69G  37% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  812M  59% /sbin/docker-init
tmpfs           6.4G   36K  6.4G   1% /var/colab
/dev/sda1        81G   43G   38G  54% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           108G   49G   60G  46% /content/drive


In [ ]:
cd /content

/content


In [ ]:
!nvidia-smi; free -h

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

              total        used        free      shared  buff/cache   available
Mem:            12G        734M        7.7G        1.2M        4.3G         11G
Swap:            0B          0B          0B


In [ ]:
import os

model_flags = ['hidden_size', 'ff_size', 'heads', 'emb_size', 'enc_layers', 'enc_hidden_size', 'enc_ff_size',
               'dec_layers', 'dec_hidden_size', 'dec_ff_size', 'encoder', 'ff_actv', 'use_interval']

BERT_DATA_PATH='./drive/MyDrive/ColabNotebooks/dacon-gas/'
OUTPUT_PATH='./drive/MyDrive/ColabNotebooks/dacon-gas/bertsumabs/testest'
CHECKPOINT_PATH='./drive/MyDrive/ColabNotebooks/dacon-gas/pre/checkpoint/model_step_200000.pt'
LOGDIR='./logs/news.log'

args = easydict.EasyDict({
    "task": 'ext',
    "encoder": 'bert',
    "mode": 'train',
    "bert_data_path": './bert_data_kor/',
    "model_path": './models/',
    "result_path": './results/news',
    "temp_dir": './temp',

    "batch_size": 140,
    "test_batch_size": 200,

    "max_pos": 512,
    "use_interval":True,
    "large":False,
    "load_from_extractive": '',

    "sep_optim": False,
    "lr_bert": 2e-3,
    "lr_dec": 2e-3,
    "use_bert_emb": False,

    "share_emb":False,
    "finetune_bert": True,
    "dec_dropout": 0.2,
    "dec_layers": 6,
    "dec_hidden_size": 768 ,
    "dec_heads": 8 ,
    "dec_ff_size": 2048 ,
    "enc_hidden_size": 512 ,
    "enc_ff_size": 512 ,
    "enc_dropout": 0.2 ,
    "enc_layers": 6 ,

    # params for EXT
    "ext_dropout": 0.2 ,
    "ext_layers": 2 ,
    "ext_hidden_size": 768 ,
    "ext_heads": 8 ,
    "ext_ff_size": 2048 ,

    "label_smoothing": 0.1 ,
    "generator_shard_size": 32 ,
    "alpha":  0.6 ,
    "beam_size": 5 ,
    "min_length": 15 ,
    "max_length": 150 ,
    "max_tgt_len": 140 ,



    "param_init": 0 ,
    "param_init_glorot": True,
    "optim": 'adam' ,
    "lr": 1 ,
    "beta1":  0.9 ,
    "beta2": 0.999 ,
    "warmup_steps": 8000 ,
    "warmup_steps_bert": 8000 ,
    "warmup_steps_dec": 8000 ,
    "max_grad_norm": 0 ,

    "save_checkpoint_steps": 5 ,
    "accum_count": 1 ,
    "report_every": 1 ,
    "train_steps": 1000 ,
    "recall_eval":False,


    'visible_gpus': '-1' ,
    'gpu_ranks': '-1' ,
    'log_file': './logs/cnndm.log',
    'seed': 666 ,

    "test_all": False,
    "test_from": '',
    "test_start_from": -1 ,

    "train_from": '',
    "report_rouge": True,
    "block_trigram": True,
})

args.world_size = len(args.gpu_ranks)
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

In [ ]:
args.task = 'abs'
args.mode = 'test'
args.batch_size = 3000
args.test_batch_size = 3000
args.bert_data_path = BERT_DATA_PATH
args.log_file = LOGDIR
args.sep_optim = True
args.use_interval = True
args.visible_gpus = '-1'
args.max_pos = 512
args.max_tgt_len = 300
args.alpha = 0.95
args.min_length = 50
args.report_rouge = False
args.result_path = OUTPUT_PATH
args.test_from = CHECKPOINT_PATH

In [ ]:
!git clone https://github.com/hyunwoongko/kss

fatal: destination path 'kss' already exists and is not an empty directory.


In [ ]:
cd /content/kss

/content/kss


In [ ]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/kss
  Attempting uninstall: kss
    Found existing installation: kss 3.4.3
    Can't uninstall 'kss'. No files were found to uninstall.
  Running setup.py develop for kss


In [ ]:
cd /content

/content


In [ ]:
# !pip install -v python-mecab-ko==1.0.9

In [ ]:
!pip install -q jupyter-dash==0.3.0 dash-bootstrap-components

In [ ]:
!pip install -q dash==2.0.0

****

In [ ]:
from train_abstractive import test_abs
from prepro.data_builder import BertData

In [ ]:
ddd = BertData().preprocess('''
[헤럴드경제=서정은 기자] 우리금융그룹이 올 상반기 1조7000억원대의 순이익을 거두며 호실적 잔치를 이어갔다. 타 금융그룹과 마찬가지로 은행이 급성장한 가운데, 우리금융캐피탈이 전년 동기대비 50% 이상 성장하며 숨은 활약을 했다.

우리금융그룹은 22일 실적발표를 통해 상반기 1조7614억원의 순이익을 시현했다고 밝혔다. 국내외 경기 하방위험 등에도 견고해진 수익창출력을 입증했다는 설명이다.

상반기 순영업수익은 수익기반 다변화에 힘입어 전년 동기 대비 20.8% 증가한 4조8861억원을 기록했다. 기업대출 중심 자산 성장을 이룬 가운데 기업투자금융(CIB, CB+IB)역량 강화에 따른 투자은행(IB)부분 손익 확대 등이 순익 증가에 기여를 했다.

이에 따라 자연스럽게 수익에서 비은행부문이 차지하는 비중도 늘었다. 지주 설립시 10% 수준이었던 비은행부문 수익 비중은 이번 상반기를 계기로 20% 수준까지 올랐다.

이밖에 고정이하여신(NPL) 비율 0.30%, 연체율 0.21%로 지난 분기에 이어 안정적 수준을 유지했으며 우량자산비율과 NPL커버리지비율도 각각 89.6%, 210.3%를 기록했다.

우리금융그룹 또한 2분기 미래경기전망을 반영해 충당금 1310억원을 추가 적립했다. 대외여건 악화시 충분한 대응 능력을 확보하라는 금융당국의 기조에 발맞춘 것으로 풀이된다.

손태승 우리금융 회장은 투자자 메시지를 통해 3고(高)를 기록 중인 물가·환율·금리 움직임에 따른 복합경제위기 가능성에 대비 적극적 리스크 관리로 안정적 성장을 이어나가겠다는 뜻을 내비쳤다. 아울러 이번 중간배당(주당150원)을 포함해 다양한 주주환원활동도 추진할 예정이다.

자회사별로 보면 단연 순이익 핵심 효자는 우리은행이었다. 우리은행의 당기순이익은 1조5545억원으로 전년 동기대비 21% 상승했다. 금리상승에 따라 순이자마진(NIM)도 지난 상반기 1.37%에서 올 상반기 1.58%까지 올랐다. 총자산규모는 448조8000억원으로 전년 말 대비 7.9% 늘었다.

이밖에 우리카드는 1343억원의 순이익의 실적을 거두며 전년 동기대비 10.6% 성장했다. 우리금융캐피탈은 1249억원의 순이익을 기록, 51.9%의 성장세를 보였다. 우리금융캐피탈 관계자는 “2020년말 편입된 뒤 급격한 자산성장을 보이다보니까 수익성이 좋아진 것”이라고 설명했다. 우리금융캐피탈의 대출자산 규모는 11조5700억원으로 지난해 상반기에 비해 43.1%가 뛰었다.

우리종합금융 또한 453억원의 순이익을 기록하며 전년 동기 대비 3% 성장했다. KB금융, 신한금융그룹이 증권업에서 순이익 급감을 기록하며 부진했던 가운데, 증권사가 없는 우리금융이 비교적 영향을 비켜갔다. 우리종합금융의 투자은행(IB) 관련이익은 올 상반기 350억원으로 전년 동기에 비해 31.4%가 뛰었다.
''')

using cached model. /content/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
pred = test_abs(args, ddd, device_id='-1', pt='', step=200000)

{'task': 'abs', 'encoder': 'bert', 'mode': 'test', 'bert_data_path': './drive/MyDrive/ColabNotebooks/dacon-gas/', 'model_path': './models/', 'result_path': './drive/MyDrive/ColabNotebooks/dacon-gas/bertsumabs/testest', 'temp_dir': './temp', 'batch_size': 3000, 'test_batch_size': 3000, 'max_pos': 512, 'use_interval': True, 'large': False, 'load_from_extractive': '', 'sep_optim': True, 'lr_bert': 0.002, 'lr_dec': 0.002, 'use_bert_emb': False, 'share_emb': False, 'finetune_bert': True, 'dec_dropout': 0.2, 'dec_layers': 6, 'dec_hidden_size': 768, 'dec_heads': 8, 'dec_ff_size': 2048, 'enc_hidden_size': 512, 'enc_ff_size': 512, 'enc_dropout': 0.2, 'enc_layers': 6, 'ext_dropout': 0.2, 'ext_layers': 2, 'ext_hidden_size': 768, 'ext_heads': 8, 'ext_ff_size': 2048, 'label_smoothing': 0.1, 'generator_shard_size': 32, 'alpha': 0.95, 'beam_size': 5, 'min_length': 50, 'max_length': 150, 'max_tgt_len': 300, 'param_init': 0, 'param_init_glorot': True, 'optim': 'adam', 'lr': 1, 'beta1': 0.9, 'beta2': 0.

In [ ]:
pred

['우리금융그룹은 22일 실적발표에서 상반기 순영업수익은 전년 동기 대비 20 . 8 % 증가한 4조8861억원을 기록하였으며 , 이는 전년 동기 대비 50 % 이상 성장하며 숨은 활약을 한 것이다 .']

In [ ]:
import time

import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import torch
def textbox(text, box="other"):
    style = {
        "max-width": "55%",
        "width": "max-content",
        "padding": "10px 15px",
        "border-radius": "25px"
    }

    if box == "self":
        style["margin-left"] = "auto"
        style["margin-right"] = 0

        color = "primary"
        inverse = True

    elif box == "other":
        style["margin-left"] = 0
        style["margin-right"] = "auto"

        color = "light"
        inverse = False

    else:
        raise ValueError("Incorrect option for `box`.")

    return dbc.Card(text, style=style, body=True, color=color, inverse=inverse)
conversation = html.Div(
    style={
        "width": "80%",
        "max-width": "800px",
        "height": "70vh",
        "margin": "auto",
        "overflow-y": "auto",
    },
    id="display-conversation",
)

controls = dbc.InputGroup(
    style={"width": "80%", "max-width": "800px", "margin": "auto"},
    children=[
        dbc.Input(id="user-input", placeholder="Write to the chatbot...", type="text"),
        dbc.Button("Submit", id="submit"),
    ],
)


# Define app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server


# Define Layout
app.layout = dbc.Container(
    fluid=True,
    style={'background-image': 'url(https://cdn.pixabay.com/photo/2020/05/19/13/54/man-5190962_960_720.jpg)'},
    children=[
        html.H1("BertAbsSum"),
        html.Hr(),
        dcc.Store(id="store-conversation", data=""),
        conversation,
        controls
    ],
)
@app.callback(
    Output("display-conversation", "children"), [Input("store-conversation", "data")]
)
def update_display(chat_history):
    return [
        textbox(x, box="self") if i % 2 == 0 else textbox(x, box="other")
        for i, x in enumerate(chat_history.split('<token>'))
    ]


@app.callback(
    [Output("store-conversation", "data"), Output("user-input", "value")],
    [Input("submit", "n_clicks"), Input("user-input", "n_submit")],
    [State("user-input", "value"), State("store-conversation", "data")],
)
def run_chatbot(n_clicks, n_submit, user_input, chat_history):
    if n_clicks == 0:
        return "", ""

    if user_input is None or user_input == "":
        return chat_history, ""

    bot_input_ids = BertData().preprocess(user_input)
        
    final_text = test_abs(args, bot_input_ids, -1, '', 200000)
    chat_history = user_input + '<token>' +final_text[0]
    return chat_history, ""

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """


In [ ]:
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>